In [1]:
import socket,cv2, pickle,struct, threading, imutils

# create socket
client_socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
host_ip = '192.168.43.195'
port = 9999
client_socket.connect((host_ip,port)) # a tuple

        

**Receiving Video from Server**

In [2]:
def recv():
    data = b""
    payload_size = struct.calcsize("Q")
    while True:
        while len(data) < payload_size:
            packet = client_socket.recv(4*1024) # 4K
            if not packet: break
            data+=packet
        packed_msg_size = data[:payload_size]
        data = data[payload_size:]
        msg_size = struct.unpack("Q",packed_msg_size)[0]

        while len(data) < msg_size:
            data += client_socket.recv(4*1024)
        frame_data = data[:msg_size]
        data  = data[msg_size:]
        frame = pickle.loads(frame_data)
        cv2.imshow("Server to client video",frame)
        if cv2.waitKey(1)  == 13:
            break
    cv2.destroyAllWindows()
    client_socket.close()

**Streaming Video to Server**

In [3]:
def sendm():
    if client_socket:
        vid = cv2.VideoCapture(1)

        while(vid.isOpened()):
            try:
                img,frame = vid.read()
                frame = imutils.resize(frame,width=320)
                a = pickle.dumps(frame)
                message = struct.pack("Q",len(a))+a
                client_socket.sendall(message)
            except ConnectionAbortedError:
                vid.release()
                client_socket.close()
                break

In [4]:
t1=threading.Thread(target=recv)
t1.start()
t2=threading.Thread(target=sendm)
t2.start()

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\hp\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\hp\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-739b2f3d93a2>", line 11, in recv
    msg_size = struct.unpack("Q",packed_msg_size)[0]
struct.error: unpack requires a buffer of 8 bytes

